In [153]:
import utils
import data_utils as du
import os
import pandas as pd
import pickle
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd

# Get input data

# Unzip pickles.zip

In [ ]:
inputs = du.cleaned_longitudinal_inputs(load_pkl=False)

# Get X and Y sequences and datasets, Train, Evaluate

In [ ]:
# import data_utils as du
# lookback = 24
# lookfwd = 8
# seqgen = du.IBDSequenceGenerator(lookback, lookfwd)
# dir=f'm{lookback}p{lookfwd}'
# seqgen.write_to_disk(datadir=dir)
# du.get_tt_datasets(datadir=dir, dataset_name=f'{dir}_ds', train_size=0.8, stratify_by='annual_charges')

"import data_utils as du;lookback = 18;lookfwd = 6;seqgen = du.IBDSequenceGenerator(lookback, lookfwd);dir=f'm{lookback}p{lookfwd}';seqgen.write_to_disk(datadir=dir);du.get_tt_datasets(datadir=dir, dataset_name=f'{dir}_ds', train_size=0.7, stratify_by='annual_charges')"


In [ ]:
python train.py -b128 -Tannual_charges -Aabnormal_labs,diagnostics,surgery -a0.9 -t0.6 -dm18p6_ds -m18m6m -e300

In [ ]:
python evaluate.py -m2y8m_4 -Tannual_charges -o3 -dm24p8_ds 

# Evaluate

## Get sequences and their predictions

In [168]:
import utils 
import numpy as np
import pandas as pd

for MODEL in ['2y8m_3', '2y8m_4']:
    def data(ix):
        d = test_results.loc[ix:ix+32]
        d['0'] = 'TEST'
        d['0'].iloc[:24] = 'TRAIN'
        return d


    ds = utils.pkl_load('datasets/m24p8_ds/test.dataset')
    test_ids = pd.Series(ds.list_IDs)
    results = pd.DataFrame(np.array(utils.pkl_load(f'models/{MODEL}/test_eval.pkl')))
    results.columns = ['y', 'yhat', 'logit0', 'logit1', 'logit2']
    results['seq_id'] = test_ids

    test_input = pd.read_pickle('pickles/x_padded_inputs.pkl') 
    test_input['seq_id'] = test_input.AUTO_ID.astype(str)+'_'+test_input.MONTH_TS.astype(str)

    test_results = results.merge(test_input, how='outer', on='seq_id').sort_values('seq_id')
    test_results = test_results.sort_values(['AUTO_ID','MONTH_TS'])
    test_results.index = range(len(test_results))

    test_results['tag'] = ''
    test_results.loc[(test_results['y']==2) & (test_results['yhat']==0), 'tag'] = 'fn'
    test_results.loc[(test_results.y==2)&(test_results.yhat==2), 'tag'] = 'tp'
    test_results.loc[(test_results.y==0)&(test_results.yhat==0), 'tag'] = 'tn'
    test_results.loc[(test_results.y==0)&(test_results.yhat==2), 'tag'] = 'fp'

    print(test_results.tag.value_counts())
    for i in ['fn', 'tp', 'tn', 'fp']:
        dfs = [data(ix) for ix in test_results[test_results.tag==i].index]
        pd.concat(dfs, axis=0).to_csv(f'models/patient_seq/{MODEL}/{i}.csv')

229200
tn     12939
tp       941
fp       393
fn        76
Name: tag, dtype: int64
      229200
tn     12939
tp       941
fp       393
fn        76
Name: tag, dtype: int64


In [ ]:
# Get TP/TN/FN/FP sequences
dfs = [data(ix) for ix in test_results[test_results.tag=='fn'].index]
pd.concat(dfs, axis=0).to_csv(f'models/patient_seq/{MODEL}/FalseNegatives.csv')

## Classification Report

In [165]:
from sklearn.metrics import classification_report, roc_curve, auc
from pprint import pprint
from scipy.special import softmax
import torch

for MODEL in ['2y8m_3', '2y8m_4']:
    ds = utils.pkl_load('datasets/m24p8_ds/test.dataset')
    test_ids = pd.Series(ds.list_IDs)
    results = pd.DataFrame(np.array(utils.pkl_load(f'models/{MODEL}/test_eval.pkl')))
    results.columns = ['y', 'yhat', 'logit0', 'logit1', 'logit2']
    results['seq_id'] = test_ids

    report = classification_report(results.y, results.yhat, labels=[0,1,2], target_names=['lt_7k', 'lt_53k', 'gt_54k'], output_dict=True)

    y_dum = pd.get_dummies(results.y).values
    y_hats = softmax(results[['logit0', 'logit1', 'logit2']].values, axis=0)
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(3):
        fpr[i], tpr[i], _ = roc_curve(y_dum[:, i], y_hats[:, i])
        roc_auc[i] = round(auc(fpr[i], tpr[i]), 4)

    print(f"Model: {MODEL}")
    print(f"Accuracy:{report['accuracy']}\nMultilclass ROC:{roc_auc}\n")
    del report['accuracy']
    print(pd.DataFrame.from_dict(report))
    print("\n=====================================================================================\n")


Model: 2y8m_3
Accuracy:0.9173595049071701
Multilclass ROC:{0: 0.8892, 1: 0.7774, 2: 0.9364}

                  lt_7k       lt_53k       gt_54k     macro avg  weighted avg
precision      0.981789     0.566239     0.665488      0.737839      0.933643
recall         0.931668     0.765159     0.879439      0.858755      0.917360
f1-score       0.956072     0.650839     0.757649      0.788187      0.922975
support    13888.000000  1039.000000  1070.000000  15997.000000  15997.000000


Model: 2y8m_4
Accuracy:0.9173595049071701
Multilclass ROC:{0: 0.8892, 1: 0.7774, 2: 0.9364}

                  lt_7k       lt_53k       gt_54k     macro avg  weighted avg
precision      0.981789     0.566239     0.665488      0.737839      0.933643
recall         0.931668     0.765159     0.879439      0.858755      0.917360
f1-score       0.956072     0.650839     0.757649      0.788187      0.922975
support    13888.000000  1039.000000  1070.000000  15997.000000  15997.000000


